In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import sklearn 
from keras.layers import Dense
from keras.models import Sequential
# from google.colab import drive
# drive.mount('/content/gdrive')
import os

In [2]:
df = pd.read_csv('/home/saichaitanya/Chaitanya/CSV files/1000 points per trajectory/May10rd1percent_RLV_data 1000 points.csv',header = None,names = ['h','v','s','omega','gamma','m','theta','Thrust','beta','time'])
input = output =df.values
X = input[:,0:7]
y = output[:,7:9]
# Individual Data
from sklearn import preprocessing
X_norm = preprocessing.minmax_scale(X)
y_norm = preprocessing.minmax_scale(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_norm, y_norm, test_size=0.1, random_state=42)
X_val = X_train[-10000:]
y_val = y_train[-10000:]
X_train = X_train[:-10000]
y_train = y_train[:-10000]

In [3]:
def create_model2(n):
    model = Sequential()
    model.add(Dense(n,input_shape=(7,),kernel_initializer='uniform'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu')) # since tanh has more nonlinearity we add it here, it also gives -ve values so , some layers which are not necessary will lead to 0 in next layer
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(n,kernel_initializer='uniform',activation = 'relu'))
    model.add(Dense(2,kernel_initializer='uniform',activation = 'sigmoid'))
    return model

In [4]:
i = 7;
model = create_model2(2**i)
opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
loss = tf.keras.losses.mean_squared_error
def lr_sch3(epoch,lr):
    if (epoch>0)& (epoch<100):
      if epoch%30==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>100):
        if epoch%20==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)
print("The Number of Hidden units used is: ",2**i)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_sch3,verbose = 1)
bs = 256;
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 5
checkpoint_path = f"/home/saichaitanya/Chaitanya/GOOGLE COLAB  DOCS/1new percent variation/1000 points 7 states and 2 op with val_set having 128 neurons and batchsize 256/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')
iter1 = 150

The Number of Hidden units used is:  128


2022-05-13 19:39:21.955314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-05-13 19:39:21.955361: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-05-13 19:39:21.956037: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
STEPS_PER_EPOCH = X_train.shape[0] / bs
save_period = 20
int(save_period*STEPS_PER_EPOCH)/7658

19.998041264037607

In [6]:
hist = model.fit(X_train,y_train,epochs = iter1,batch_size = bs,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/150
7658/7658 [==============================] - 50s 6ms/step - loss: 0.0054 - accuracy: 0.9183 - val_loss: 0.0033 - val_accuracy: 0.9326 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/150
7658/7658 [==============================] - 48s 6ms/step - loss: 0.0031 - accuracy: 0.9382 - val_loss: 0.0027 - val_accuracy: 0.9367 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/150
7658/7658 [==============================] - 47s 6ms/step - loss: 0.0026 - accuracy: 0.9442 - val_loss: 0.0028 - val_accuracy: 0.9337 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/150
7658/7658 [==============================] - 47s 6ms/step - loss: 0.0024 - accuracy: 0.9474 - val_loss: 0.0028 - val_accuracy: 0.9417 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.001.
Epoch 5/150
7652/7658 [=====================

7658/7658 [==============================] - 38s 5ms/step - loss: 0.0012 - accuracy: 0.9657 - val_loss: 5.9800e-04 - val_accuracy: 0.9786 - lr: 0.0010

Epoch 31: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 31/150
7658/7658 [==============================] - 39s 5ms/step - loss: 6.8669e-04 - accuracy: 0.9749 - val_loss: 3.5322e-04 - val_accuracy: 0.9799 - lr: 6.3760e-04

Epoch 32: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 32/150
7658/7658 [==============================] - 39s 5ms/step - loss: 6.3362e-04 - accuracy: 0.9756 - val_loss: 0.0011 - val_accuracy: 0.9679 - lr: 6.3760e-04

Epoch 33: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 33/150
7658/7658 [==============================] - 38s 5ms/step - loss: 7.1418e-04 - accuracy: 0.9746 - val_loss: 3.4760e-04 - val_accuracy: 0.9834 - lr: 6.3760e-04

Epoch 34: LearningRateScheduler setting learning rate to 0.0006376.
Epoch 34/150
7658/7658 [==============================] - 38s

7658/7658 [==============================] - 48s 6ms/step - loss: 4.0299e-04 - accuracy: 0.9819 - val_loss: 4.8632e-04 - val_accuracy: 0.9800 - lr: 4.0660e-04

Epoch 86: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 86/150
7658/7658 [==============================] - 49s 6ms/step - loss: 3.8056e-04 - accuracy: 0.9822 - val_loss: 0.0010 - val_accuracy: 0.9684 - lr: 4.0660e-04

Epoch 87: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 87/150
7658/7658 [==============================] - 48s 6ms/step - loss: 3.8373e-04 - accuracy: 0.9823 - val_loss: 2.7334e-04 - val_accuracy: 0.9844 - lr: 4.0660e-04

Epoch 88: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 88/150
7658/7658 [==============================] - 49s 6ms/step - loss: 3.7160e-04 - accuracy: 0.9825 - val_loss: 2.0795e-04 - val_accuracy: 0.9875 - lr: 4.0660e-04

Epoch 89: LearningRateScheduler setting learning rate to 0.0004066.
Epoch 89/150
7658/7658 [=============================

7658/7658 [==============================] - 48s 6ms/step - loss: 2.2240e-04 - accuracy: 0.9872 - val_loss: 2.2102e-04 - val_accuracy: 0.9866 - lr: 2.2040e-04

Epoch 141: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 141/150
7658/7658 [==============================] - 48s 6ms/step - loss: 1.9119e-04 - accuracy: 0.9884 - val_loss: 1.4986e-04 - val_accuracy: 0.9886 - lr: 1.8734e-04

Epoch 142: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 142/150
7658/7658 [==============================] - 48s 6ms/step - loss: 1.7540e-04 - accuracy: 0.9891 - val_loss: 1.0679e-04 - val_accuracy: 0.9921 - lr: 1.8734e-04

Epoch 143: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 143/150
7658/7658 [==============================] - 49s 6ms/step - loss: 1.9458e-04 - accuracy: 0.9883 - val_loss: 1.2269e-04 - val_accuracy: 0.9913 - lr: 1.8734e-04

Epoch 144: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 144/150
7658/7658 [=========

In [9]:
lr = hist.history.get('lr')[-1]
opt = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.mean_squared_error
iter2 = 125
def lr_sch(epoch,lr):
    if (epoch >0) &(epoch<25):
      if epoch%5==0: # for every 100 epochs the learning rate varies as metioned. 
        return round(lr*np.exp(-0.45),7)
      else:
        return round(lr,7)
    elif(epoch>25):
        if epoch%5==0:
            return round(lr*0.85,9)
        else:
            return round(lr,9)
    else:
        return round(lr,9)

bs2 = 1024
STEPS_PER_EPOCH = X_train.shape[0] / bs2
save_period = 5
print("The Number of Hidden units used is: ",2**i)
lr_scheduler2 = tf.keras.callbacks.LearningRateScheduler(lr_sch,verbose = 1)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1,
                                            save_freq=int(save_period*STEPS_PER_EPOCH))

model.load_weights(checkpoint_path)
model.compile(optimizer = opt, loss = loss, metrics = 'accuracy')

The Number of Hidden units used is:  128


In [10]:
hist = model.fit(X_train,y_train,epochs = iter2,batch_size = bs2,shuffle = True,use_multiprocessing = True,callbacks=[lr_scheduler2,cp_callback],validation_data=(X_val, y_val))


Epoch 1: LearningRateScheduler setting learning rate to 0.000187344.
Epoch 1/125
1915/1915 [==============================] - 17s 8ms/step - loss: 1.2933e-04 - accuracy: 0.9911 - val_loss: 6.8576e-05 - val_accuracy: 0.9943 - lr: 1.8734e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 2/125
1915/1915 [==============================] - 16s 8ms/step - loss: 1.1422e-04 - accuracy: 0.9915 - val_loss: 1.1579e-04 - val_accuracy: 0.9900 - lr: 1.8730e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 3/125
1915/1915 [==============================] - 16s 8ms/step - loss: 1.4233e-04 - accuracy: 0.9905 - val_loss: 9.9148e-05 - val_accuracy: 0.9901 - lr: 1.8730e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.0001873.
Epoch 4/125
1915/1915 [==============================] - 16s 8ms/step - loss: 1.4560e-04 - accuracy: 0.9902 - val_loss: 8.7087e-05 - val_accuracy: 0.9910 - lr: 1.8730e-04

Epoch 5: LearningRateScheduler setting le

1915/1915 [==============================] - 16s 8ms/step - loss: 4.5762e-05 - accuracy: 0.9958 - val_loss: 3.6085e-05 - val_accuracy: 0.9955 - lr: 1.1653e-05

Epoch 57: LearningRateScheduler setting learning rate to 1.1653e-05.
Epoch 57/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.5101e-05 - accuracy: 0.9958 - val_loss: 3.6546e-05 - val_accuracy: 0.9956 - lr: 1.1653e-05

Epoch 58: LearningRateScheduler setting learning rate to 1.1653e-05.
Epoch 58/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.5578e-05 - accuracy: 0.9958 - val_loss: 5.1389e-05 - val_accuracy: 0.9949 - lr: 1.1653e-05

Epoch 59: LearningRateScheduler setting learning rate to 1.1653e-05.
Epoch 59/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.5170e-05 - accuracy: 0.9958 - val_loss: 3.6135e-05 - val_accuracy: 0.9958 - lr: 1.1653e-05

Epoch 60: LearningRateScheduler setting learning rate to 1.1653e-05.
Epoch 60/125
1865/1915 [=====================

1915/1915 [==============================] - 16s 8ms/step - loss: 4.1238e-05 - accuracy: 0.9961 - val_loss: 3.3462e-05 - val_accuracy: 0.9961 - lr: 1.9510e-06

Epoch 112: LearningRateScheduler setting learning rate to 1.951e-06.
Epoch 112/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.1212e-05 - accuracy: 0.9961 - val_loss: 3.3807e-05 - val_accuracy: 0.9959 - lr: 1.9510e-06

Epoch 113: LearningRateScheduler setting learning rate to 1.951e-06.
Epoch 113/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.1157e-05 - accuracy: 0.9962 - val_loss: 3.3607e-05 - val_accuracy: 0.9962 - lr: 1.9510e-06

Epoch 114: LearningRateScheduler setting learning rate to 1.951e-06.
Epoch 114/125
1915/1915 [==============================] - 16s 8ms/step - loss: 4.1178e-05 - accuracy: 0.9962 - val_loss: 3.4563e-05 - val_accuracy: 0.9960 - lr: 1.9510e-06

Epoch 115: LearningRateScheduler setting learning rate to 1.951e-06.
Epoch 115/125
1818/1915 [=================